In [29]:
from colorama import Fore

In [14]:
!pip install --user xlrd

In [15]:
!tree -h --sort=size 開放權威檔/DDBC-Place

!tree -h --sort=size 開放權威檔/CBDB-Place

# Setup functions

In [181]:
import pandas as pd
import os
import re

_filter_empty = lambda s: s != ''
filter_empty = lambda l: list(filter(_filter_empty , l))
filter_empty(re.split('[()]', '某氏(徐庶母)'))

def split_aliases(names):
    results = []
    for name in names:
        try:
            results.extend(filter_empty(re.split('[=()（）]', name)))
        except TypeError:
#             print('TypeError: value - ', name)
            pass
    return results

def get_entities(data_dirpath, col_name):
    all_entities = []

    for fname in os.listdir(data_dirpath):
        fpath = os.path.join(data_dirpath, fname)
        if os.path.isdir(fpath):
            print(fpath, 'is a directory. Go inside!')
            all_entities.extend(get_entities(fpath, col_name))
#             pass
        elif not os.path.isfile(fpath):
            print(fpath, 'is not a file. Skipped')
            continue
        else:
#             print('Processing', fpath, '...')
            df = pd.read_excel(fpath)
            entities = df[col_name].to_list()
            all_entities.extend(split_aliases(entities))
    return all_entities

def filter_symbols(es):
    special_symbols = '！？!?［］[]{}｛｝＠＃＄％︿＆＊＿＋｜＂：﹀＜～／－~@#$%^&*_-+=\\|"\';:></'

    f = lambda e: not (set(e) & set(special_symbols))
    return list(filter(f, es))

def main(data_dirpath, col_name, segmenter_output_fname, regexner_output_fname, tag, overridden_tag, min_len=2):
    all_entities = get_entities(data_dirpath, col_name)
    print('(info) total', len(all_entities))
    all_entities = filter_symbols(all_entities)
    print('(proc) after removing entites with symbols:', len(all_entities))
    all_entities = list(set(all_entities))
    print('(proc) after removing duplicates:', len(all_entities))
    from collections import Counter
    print('(info)', Counter(map(len, all_entities)))
    df = pd.DataFrame({"entity": all_entities, "tag": [tag] * len(all_entities), "override": [overridden_tag] * len(all_entities)})
    print('(proc) built data frame', df.head())
    print('(info) long entity', df.query('entity.str.len() > 5').head())
    
    if min_len:
        print(f'(proc) removing length < {min_len}:', df.query(f'entity.str.len() < {min_len}').head())
        df = df.query(f'entity.str.len() >= {min_len}')
        print('(info)', f'after removing length < {min_len}:', len(df))

    df.to_csv(regexner_output_fname, header=None, index=None, sep='\t')
    print(f'(proc) saving file to {regexner_output_fname}...')
    df.to_csv(segmenter_output_fname, header=None, index=None, columns=['entity'], sep='\t')
    print(f'(proc) saving file to {segmenter_output_fname}...')
    
    # copy to inside docker container
    print(f'(proc) copying {regexner_output_fname} to inside docker container ...')
    !docker cp {regexner_output_fname} corenlp_zh:/stanford-corenlp-full-2018-10-05


    print(f'(proc) copying {segmenter_output_fname} to inside docker container ...')
    !docker cp {segmenter_output_fname} corenlp_zh:/stanford-corenlp-full-2018-10-05
        
    # check
    print('(check) if existing in local ...')
    !head -n 2 {regexner_output_fname}
    !wc -l {regexner_output_fname}

    !head -n 2 {segmenter_output_fname}
    !wc -l {segmenter_output_fname}
    
    # check in docker
    print('(check) if existing in docker container ...')
    !docker exec corenlp_zh head -n 2 {regexner_output_fname}
    !docker exec corenlp_zh wc -l {regexner_output_fname}

    !docker exec corenlp_zh head -n 2 {segmenter_output_fname}
    !docker exec corenlp_zh wc -l {segmenter_output_fname}

# Main Process

In [184]:
root_dir = '開放權威檔'

## CBDB-Place

# input
col_name = '行政區'
data_dir = 'CBDB-Place'
regexner_output_fname = 'cbdb-gpe-regexner.txt'
segmenter_output_fname = 'cbdb-gpe-segmenter.txt'
tag = 'GPE'
overridden_tag = 'O,PERSON'

data_dirpath = os.path.join(root_dir, data_dir)
main(data_dirpath, col_name, segmenter_output_fname, regexner_output_fname, tag, overridden_tag)

## DDBC-Place

col_name = '名'
data_dir = 'DDBC-Place'
regexner_output_fname = 'ddbc-place-regexner.txt'
segmenter_output_fname = 'ddbc-place-segmenter.txt'
tag = 'LOCATION'
overridden_tag = 'O,PERSON'
data_dirpath = os.path.join(root_dir, data_dir)
main(data_dirpath, col_name, segmenter_output_fname, regexner_output_fname, tag, overridden_tag)

## Generate properties file

In [124]:
regexner_files = ['cbdb-person-regexner.txt', 'ddbc-person-regexner.txt', 'cbdb-gpe-regexner.txt', 'ddbc-place-regexner.txt']
segmenter_files = ['cbdb-person-segmenter.txt', 'ddbc-person-segmenter.txt', 'cbdb-gpe-segmenter.txt', 'ddbc-place-segmenter.txt']

In [193]:
props_inpath = 'StanfordCoreNLP-chinese-fgc-template.properties'
props_outpath = 'StanfordCoreNLP-chinese-fgc.properties'

print('(proc) generating properties file ...')
!cp {props_inpath} {props_outpath}

old_str = '#ner.additional.regexner.mapping = ...'
new_str = 'ner.additional.regexner.mapping = ' + ','.join(regexner_files)
!sed -i 's/{old_str}/{new_str}/g' {props_outpath}

old_str = 'segment.serDictionary = edu\/stanford\/nlp\/models\/segmenter\/chinese\/dict-chris6.ser.gz'
new_str = old_str + ',' + ','.join(segmenter_files)
!sed -i 's/{old_str}/{new_str}/g' {props_outpath}

print('(modified part)')
!diff --color {props_inpath} {props_outpath}

print('(proc) moving to docker container')
!docker cp {props_outpath} corenlp_zh:/stanford-corenlp-full-2018-10-05/{props_outpath}
print('(check) if moved')
!docker exec corenlp_zh grep 'segment.serDictionary' {props_outpath}
!docker exec corenlp_zh grep 'ner.additional.regexner.mapping' {props_outpath}

# Restart CoreNLPServer

In [6]:
# kill existent server
ps_str = !docker exec corenlp_zh ps | grep StanfordCoreNLPServer
if ps_str:
    print('(proc) killing server ps:', ps_str[0])
    pid = int(ps_str[0].strip().split(' ')[0])
    p = !docker exec corenlp_zh kill {pid} && ps | grep StanfordCoreNLPServer
    assert not p, p

# start a new server
print('(proc) starting a new server')
!docker exec -d corenlp_zh ash -c "java -mx8g -cp '*' edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 -serverProperties StanfordCoreNLP-chinese-fgc.properties &> run.log" && docker exec corenlp_zh ps -a | grep StanfordCoreNLPServer

(proc) starting a new server
   11 root      0:00 java -mx8g -cp * edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 -serverProperties StanfordCoreNLP-chinese-fgc.properties


# Test Results

In [5]:
!pip install stanfordnlp

     |████████████████████████████████| 163kB 189kB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 71.6MB/s eta 0:00:01
     |████████████████████████████████| 753.4MB 6.7kB/s eta 0:00:010   |▏                               | 4.2MB 48.6MB/s eta 0:00:16


In [2]:
import sys
sys.path.append('../wiki_kb_inference')
from stanfordnlp_utils import *
from fgc_utils import *
from utils import load_json
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from stanfordnlp.server import CoreNLPClient

In [3]:
docs = load_json('../wiki_kb_inference/FGC_release_all(cn).json')

In [11]:
dids = [doc['DID'] for doc in docs]
get_doc(dids[0], docs)
# dids

{'DID': 'D001',
 'DTEXT': '蘇軾（1037年1月8日－1101年8月24日），眉州眉山（今四川省眉山市）人，北宋時著名的文學家、政治家、藝術家、醫學家。字子瞻，一字和仲，號東坡居士、鐵冠道人。嘉佑二年進士，累官至端明殿學士兼翰林學士，禮部尚書。南宋理學方熾時，加賜諡號文忠，複追贈太師。有《東坡先生大全集》及《東坡樂府》詞集傳世，宋人王宗稷收其作品，編有《蘇文忠公全集》。\n其散文、詩、詞、賦均有成就，且善書法和繪畫，是文學藝術史上的通才，也是公認韻文散文造詣皆比較傑出的大家。蘇軾的散文為唐宋四家（韓愈、柳宗元、歐蘇）之末，與唐代的古文運動發起者韓愈並稱為「韓潮蘇海」，也與歐陽修並稱「歐蘇」；更與父親蘇洵、弟蘇轍合稱「三蘇」，父子三人，同列唐宋八大家。蘇軾之詩與黃庭堅並稱「蘇黃」，又與陸游並稱「蘇陸」；其詞「以詩入詞」，首開詞壇「豪放」一派，振作了晚唐、五代以來綺靡的西崑體餘風。後世與南宋辛棄疾並稱「蘇辛」，惟蘇軾故作豪放，其實清朗；其賦亦頗有名氣，最知名者為貶謫期間借題發揮寫的前後《赤壁賦》。宋代每逢科考常出現其文命題之考試，故當時學者曰：「蘇文熟，喫羊肉、蘇文生，嚼菜羹」。藝術方面，書法名列「蘇、黃、米、蔡」北宋四大書法家（宋四家）之首；其畫則開創了湖州畫派；並在題畫文學史上佔有舉足輕重的地位。',
 'QUESTIONS': [{'QID': 'D001Q01',
   'QTYPE': '基础题',
   'QTEXT': '蘇東坡在中國歷史上，是哪一個朝代的人？',
   'SENTS': [{'text': '苏东坡在中国历史上，', 'start': 0, 'end': 10},
    {'text': '是哪一个朝代的人？', 'start': 10, 'end': 19}],
   'ANSWER': [{'ATEXT': '北宋',
     'ATOKEN': [{'text': '北宋', 'start': 40}],
     'ATEXT_CN': '北宋'}],
   'ATYPE': 'Date-Duration',
   'AMODE': 'Single-Span-Extraction',
   'ASPAN': [{'text': '苏轼', 'start': 0, 'end': 2},
    {'te

In [15]:

@interact_manual
def pretty_corenlp(did=dids):
    doc_dic = get_doc(did, docs)
    print('[New]')
    with CoreNLPClient(endpoint='http://localhost:9000', start_server=False) as nlp:
        doc = nlp.annotate(doc_dic['DTEXT_CN'], properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+'})

        for sent in doc.sentence:
            print(f'(s{sent.sentenceIndex})', end=' ')
            snp_pprint(sent, mode='custom', classes_w_color=['PERSON', 'GPE', 'LOCATION', 'MISC', 'TITLE'])
    print('-----------------------------------------------------')
    print('[Old]')
    with CoreNLPClient(endpoint='http://140.109.19.191:9000', start_server=False) as nlp:
        doc = nlp.annotate(doc_dic['DTEXT_CN'], properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
                                                            'pipelineLanguage': 'zh'})

        for sent in doc.sentence:
            print(f'(s{sent.sentenceIndex})', end=' ')
            snp_pprint(sent, mode='custom', classes_w_color=['PERSON', 'GPE', 'LOCATION', 'MISC', 'TITLE'])
        

interactive(children=(Dropdown(description='did', options=('D001', 'D002', 'D003', 'D004', 'D006', 'D007', 'D0…

In [3]:
nlp = CoreNLPClient(endpoint='http://140.109.19.191:9000', start_server=False)

doc = nlp.annotate(docs[0]['DTEXT_CN'],properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
                                                    'pipelineLanguage': 'zh'},
                  annotators='tokenize,ssplit,pos,lemma,ner,depparse,parse,coref,entitylink')

for sent in doc.sentence:
    print(f'(s{sent.sentenceIndex})', end=' ')
    snp_pprint(sent)

(s0) 

TypeError: object of type 'NoneType' has no len()

In [4]:
nlp = CoreNLPClient(endpoint='http://localhost:9000', start_server=False)

doc = nlp.annotate(docs[3]['DTEXT_CN'], properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+'})

for sent in doc.sentence:
    print(f'(s{sent.sentenceIndex})', end=' ')
    snp_pprint(sent)

(s0) 

TypeError: object of type 'NoneType' has no len()

In [68]:
nlp = CoreNLPClient(endpoint='http://140.109.19.191:9000', start_server=False)

doc = nlp.annotate(docs[3]['DTEXT_CN'], properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
                                                    'pipelineLanguage': 'zh'
#                                                  'segment.serDictionary': 'edu/stanford/nlp/models/segmenter/chinese/dict-chris6.ser.gz,ddbc-person-segmenter.txt,cbdb-person-segmenter.txt',
#                                                     'ner.additional.regexner.mapping': 'ddbc-person-regexner.txt,cbdb-person-regexner.txt'
                                                   })

for sent in doc.sentence:
    print(f'(s{sent.sentenceIndex})', end=' ')
    snp_pprint(sent)

(s0) 元 祐 元年 （ 1086年 ） ， 宋哲宗 即位 ， 高 太 皇太 后 垂帘听政 ， 回朝 任 礼部 郎中 、 中书舍人 、 翰林 学士 ， 元祐 四 年 （ 1089年 ） 拜 龙图阁 学士 ， 曾 出任 杭州 、 颍州 等 知州 职务 ， 官 至 礼部 尚书 。
(s1) 绍圣 元年 （ 1094年 ） 被 哲宗 贬谪 至 惠州 、 儋州 （ 海南岛 ） 。
(s2) 元符 三 年 （ 1100年 ） ， 宋徽宗 即位 ， 向 太后 垂帘听政 ， 下诏 让 苏轼北 还 。
(s3) 建中 靖国 元年 （ 1101年 ） ， 夏天 因 冷饮 过度 ， 下痢 不止 ， 又 误 服 黄芪 ， 结果 病情 恶化 ， 「 齿间 出血 如 蚯蚓 者 无数 」 ， 七月 二十八日 于 常州 孙氏馆 病 卒 ， 享年 六十四 岁 。
(s4) 由 弟 苏辙 归葬 于 郏县 小 峨眉山 。
(s5) 南宋 宋孝宗 追赠 谥号 「 文忠 」 。
(s6) 苏轼 疲于 应付 新旧 党争 ， 遇事 「 如 食 内有蝇 ， 吐 之 乃 已 」 ， 苏轼 既 反对 王安石 比较 急进 的 改革 措施 ， 也 不 同意 旧 党 司马光 尽 废 新法 ， 所以 虽然 新党 一直 称 苏轼为 旧 党 ， 但 其实 他 在 新旧 两 党 之间 均受 排斥 ， 仕途 坎坷 ， 时常 远 贬 外方 ， 不过 他 在 各地 居官 清正 ， 为民 兴利 除弊 ， 政绩 颇 善 ， 口碑 甚佳 ， 杭州 西湖 的 苏堤 就 是 实证 。


In [106]:
nlp = CoreNLPClient(endpoint='http://localhost:9000', start_server=False)

doc = nlp.annotate(docs[27]['DTEXT_CN'], properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
#                                                  'segment.serDictionary': 'edu/stanford/nlp/models/segmenter/chinese/dict-chris6.ser.gz,ddbc-person-segmenter.txt,cbdb-person-segmenter.txt',
#                                                     'ner.additional.regexner.mapping': 'ddbc-person-regexner.txt,cbdb-person-regexner.txt'
                                                   })

for sent in doc.sentence:
    print(f'(s{sent.sentenceIndex})', end=' ')
    snp_pprint(sent)

(s0) 「 PTT 创世神 」 杜奕瑾 创办 的 台湾 人工 智慧 实验室 推出 「 雅婷 逐字稿 」 App ， 已 在 iOS 与 Android 平台 上 线 ， 官方 表示 能 节省 至少 60% 的 听打 时间 ， 还 听 得 懂 台湾 国语 和 中英 夹杂 。
(s1) 根据 App 官方 介绍 ， 由 台湾 人工 智慧 实验室 （ AILabs ） 推出 的 「 雅婷 逐字稿 」 App 除了 能 即时 做 语音 转 文字 ， 也可以 用来 提升 听障人 沟通 效率 。
(s2) 无论是 记录 生活 大小事 ， 或是 记录 访谈 、 课堂 、 会议 内容 ， 都 可以 精准 且 快速 地 达成 原本 需要 花费 大量 时间 才 有的 逐字稿 。
(s3) 脸 书 帐号 Shar Yuan 的 台湾 人工 智慧 实验室 员工 日前 发文 说明 ， 透过 「 雅婷 逐字稿 」 ， 听障 人士 可 透过 文字 的 方式 理解 其他 人 在说 什么 ， 媒体 工作者 或 记者 能 即时 生成 访谈 或 会议 逐字稿 ， 老师 也 能 快速 生成 课程 文字档 。
(s4) 杜奕瑾 今年 初 接受 商业 周刊 采访 时 说 ， 行政院 政务 委员 唐 凤 为 提倡 政府 资讯 公开 ， 常 带 著 每 分钟 打字 高达 350 字 的 速录师 薛雅婷 为 会议 制作 逐 字稿 ， 因此 台湾 人工 智慧 实验室 把 内部 开发 的 语音 辨识 系统 取名 为 「 雅婷 一 号 」 。
(s5) 杜奕瑾 在 访谈 中 指出 ， 国际 大 厂 已 把 英文 、 中文 语音 辨识 的 应用 做得 很 好 ， 台商 机会 在于 把 本 土腔 调和 特殊 用语 做 得 更 精进 。


In [104]:
nlp = CoreNLPClient(endpoint='http://140.109.19.191:9000', start_server=False)

doc = nlp.annotate(docs[25]['DTEXT_CN'], properties={'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
                                                    'pipelineLanguage': 'zh'
#                                                  'segment.serDictionary': 'edu/stanford/nlp/models/segmenter/chinese/dict-chris6.ser.gz,ddbc-person-segmenter.txt,cbdb-person-segmenter.txt',
#                                                     'ner.additional.regexner.mapping': 'ddbc-person-regexner.txt,cbdb-person-regexner.txt'
                                                   })

for sent in doc.sentence:
    print(f'(s{sent.sentenceIndex})', end=' ')
    snp_pprint(sent)

(s0) 根据 「 悬浮 微粒 特征 对 民众 健康 影响 之 研究 」 ， 捷运 竟是 PM2.5 浓度 暴露 最 高 的 交通 工具 。
(s1) 行政院 环境 保护 署 今天 表示 ， 这 项 结果 只是 瞬间 数值 ， 平均 浓度 最 高 的 仍然 是 机车 。
(s2) 环保署 与 国卫院 国家 环境 医学 研究所 106年 度 针对 针对 捷运 、 公车 、 汽车 、 机车 、 步行 、 脚踏车 等 6 大 交通 方式 ， 进行 「 悬浮 微粒 特征 对 民众 健康 影响 之 研究 」 ， 结果 报告 指出 ， 细 悬浮 微粒 （ PM2.5 ） 浓度 暴露 排名 最高 的 竟是 捷运 。
(s3) 环保署 空气 品质 保护 及 噪音 管制 处 专门 委员 徐淑芷 接受 中央社 记者 访问 时 表示 ， 国卫 院 使用 的 方式 是 以 空气 盒子 进行 测量 ， 在 定点 容易 受到 周遭 环境 或 瞬间 污染 ， 而 呈现 高值 。
(s4) 捷运 测量 处 是 在 月台 ， 列车 进站 时 的 状态 ， 因为 车刚 进站 会 有 一 阵 风 ， 在 地下 化且 没有 闸门 完全 阻隔 的 站台 ， 粉尘 或 微粒 很 容易 因 风 而起 ， 导致 瞬间 数值 高升 ， 与 一般 理解 搭 捷运 的 观念 不 一样 ， 并不 是 一直 维持 高 浓度 的 PM2.5 。
(s5) 徐淑芷 指出 ， 虽然 测量 捷运 PM2.5 浓度 瞬间 值 容易 升高 ， 但 因为 民众 不是 长时间 曝露 ， 整体 来 看相 较于 其他 交通 工具 而言 还是 低 的 ； 捷运 公司 也 有 定期 清理 轨道 、 集尘 等 ， 降低 因 风 扬起 粉尘 的 情况 ， 在 地面 的 捷运 车站 虽然 容易 受到 外界 空气 污染源 影响 ， 但 不 会 有 列车 进站 时 因 风 瞬间 扬起 的 粉尘 。
(s6) 监测 结果 ， 以 骑乘 机车 通勤 PM2.5 暴露 量 平均 浓度 最高 ， 因 道路 行驶 时 难以 避免 接近 高 污染 车辆 ， 致使 PM2.5 暴露 量 有 瞬间 攀升 严重 污染 的 情形 ； 其次 为 公车 ， 因为 常 处于 主要 交通 要道 ， 车门 开启 时 车门 外 常 受到 摩托车 围绕 ， 使 移动 源 污染物 进入 车厢